In [1]:
from aeon.datasets.tsc_data_lists import multivariate_equal_length
import pandas as pd
import numpy as np
from sklearn import preprocessing
import sklearn
from tensorflow import keras
import tensorflow as tf
import numpy as np
import time 
from utils import geng
from utils.utils import create_directory

2023-12-27 14:58:03.144740: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-27 14:58:06.208421: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
multivariate_equal_length

{'ArticularyWordRecognition',
 'AtrialFibrillation',
 'BasicMotions',
 'Cricket',
 'DuckDuckGeese',
 'ERing',
 'EigenWorms',
 'Epilepsy',
 'EthanolConcentration',
 'FaceDetection',
 'FingerMovements',
 'HandMovementDirection',
 'Handwriting',
 'Heartbeat',
 'LSST',
 'Libras',
 'MotorImagery',
 'NATOPS',
 'PEMS-SF',
 'PenDigits',
 'PhonemeSpectra',
 'RacketSports',
 'SelfRegulationSCP1',
 'SelfRegulationSCP2',
 'StandWalkJump',
 'UWaveGestureLibrary'}

# Data process

## Example of Data

In [3]:
# from aeon.datasets import load_classification
# import numpy as np
# dataset_name = "Libras"
# archive_name="MTS"
# X, Y,meta_data = load_classification(dataset_name,return_metadata=True)
# X_reshaped = np.transpose(X, (0, 2, 1))
# X = X_reshaped
# nb_classes = len(np.unique(Y, axis=0))
# nb_classes,X.shape,Y.shape,np.unique(Y)

## Data Kailuan

In [4]:
dataset_name = "Kailuan"
archive_name="MTS"
time_series = pd.read_csv("../combined_sheets.csv")
meta_data = pd.read_csv("../135case.csv")
time_series.shape,time_series

((140088, 6),
                   Timestamp  Sleepstate  Breath  Heartrate  Movement  Source
 0       2021-07-02 16:35:25           4      16         78         2       1
 1       2021-07-02 16:36:25           4      16         76         0       1
 2       2021-07-02 16:37:25           4      16         74         0       1
 3       2021-07-02 16:38:25           4      16         74         0       1
 4       2021-07-02 16:39:25           4      16         73         0       1
 ...                     ...         ...     ...        ...       ...     ...
 140083  2020-12-21 08:49:23           2      18         84         0     135
 140084  2020-12-21 08:50:23           2      18         84         0     135
 140085  2020-12-21 08:51:23           4      17         85         0     135
 140086  2020-12-21 08:52:23           4      18         84         0     135
 140087  2020-12-21 08:53:23           4      15         87         0     135
 
 [140088 rows x 6 columns])

In [5]:
# Movement全变成1 二值化
# time_series['Movement'] = np.where(time_series['Movement'] != 0, 1, 0)

## 构造每个source的二维数组

In [6]:
grouped = time_series.groupby('Source')

# 提取特征列
features = ['Sleepstate', 'Breath', 'Heartrate', 'Movement']

# # 将每个组转换为二维数组并堆叠
# three_dim_array = np.array([group[features].values for _, group in grouped])

# three_dim_array.shape # 显示三维数组的形状

## Movement 处理
- Movement 取消： 只有fold3有提升 其他4个fold下降
- Movement 全部+1 不行 都有下降


In [7]:
# time_groupby = time_series.groupby('Source')

In [8]:
# 查看movement的分布
# movement_stac=[{}]
# for i in range(1,136):
#     print(i)
#     keys, vals = np.unique(time_groupby.get_group(int(i))["Movement"].values,return_counts=True)
#     result_dict = dict(zip(keys, vals))
#     print(result_dict)
#     movement_stac.append(result_dict)

# # movement_stac

## 填充 截断 时间序列

## 填充/截断 构造数据集(去掉过少的)

In [9]:
# 重新计算每个样例的时间步数（考虑到了数据过滤）
filtered_time_steps_per_source = time_series.groupby('Source').size()

# 重新筛选出有效的 Source 过滤掉过少的
valid_sources = filtered_time_steps_per_source[filtered_time_steps_per_source > 500].index
obsolete_sources = filtered_time_steps_per_source[filtered_time_steps_per_source <= 500].index
print("valid sources:", valid_sources, "\nobsolete sources:", obsolete_sources)
# 重新获取有效的数据
valid_filtered_time_series = time_series[time_series['Source'].isin(valid_sources)]

# 重新分组
valid_filtered_grouped = valid_filtered_time_series.groupby('Source')

# 重新计算平均时间步数（此时应该没有 NaN 值）
average_time_steps = int(valid_filtered_grouped.size().mean())

print(f"\n average_time_steps:{average_time_steps}")

# 初始化新的三维数组（考虑到了有效的样例数）
valid_sources_count = len(valid_sources)
three_dim_array_valid = np.zeros((valid_sources_count, average_time_steps, len(features)))

# 填充新的三维数组
for i, (source, group) in enumerate(valid_filtered_grouped):
    data = group[features].values
    current_steps = data.shape[0]

    if current_steps < average_time_steps:
        fill_values = {
            'Breath': group['Breath'].mean(),
            'Heartrate': group['Heartrate'].mean(),
            'Sleepstate': group['Sleepstate'].median(),
            'Movement': 0
        }
        fill_array = np.array([[fill_values[feature] for feature in features]] * (average_time_steps - current_steps))
        full_data = np.vstack((data, fill_array))
    else:
        full_data = data[:average_time_steps, :]

    three_dim_array_valid[i, :, :] = full_data


final_data = three_dim_array_valid

# 进行数据标准化
final_data = geng.standard_scaler_total(final_data)

final_data.shape

valid sources: Index([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,
       ...
       125, 126, 127, 128, 129, 131, 132, 133, 134, 135],
      dtype='int64', name='Source', length=129) 
obsolete sources: Index([21, 26, 74, 75, 107, 130], dtype='int64', name='Source')

 average_time_steps:1075


(129, 1075, 4)

### 标签构造
- 意识状态
- GCS

In [10]:
labels = meta_data["意识状态"].values

valid_sources = valid_sources-1
labels = [labels[i] for i in range(len(labels)) if i in valid_sources]
labels = [1 if x == 2 else x for x in labels]
nb_classes = len(np.unique(labels, axis=0))
labels = np.array(labels)
labels, labels.shape

(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 (129,))

## (填充/截断)构造数据集 原本版 

In [11]:

# average_time_steps = int(grouped.apply(lambda x: x.shape[0]).mean())

# # 初始化三维数组
# features = ['Sleepstate', 'Breath', 'Heartrate', 'Movement']
# three_dim_array = np.zeros((135, average_time_steps, len(features)))

# # 填充三维数组
# for source, group in grouped:
#     data = group[features].values
#     current_steps = data.shape[0]

#     if current_steps < average_time_steps:
#         fill_values = {
#             'Breath': group['Breath'].mean(),
#             'Heartrate': group['Heartrate'].mean(),
#             'Sleepstate': group['Sleepstate'].median(),
#             'Movement': 0
#         }
#         fill_array = np.array([[fill_values[feature] for feature in features]] * (average_time_steps - current_steps))
#         full_data = np.vstack((data, fill_array))
#     else:
#         full_data = data[:average_time_steps, :]

#     three_dim_array[source-1, :, :] = full_data

# three_dim_array.shape  # 显示三维数组的形状


### 标签构造
- 意识状态
- GCS

In [12]:
# labels = meta_data["意识状态"].values
# nb_classes = len(np.unique(labels, axis=0))
# labels,nb_classes

In [13]:
def data_final_process(x,y):
    nb_classes = len(np.unique(y, axis=0))
    # transform the labels from integers to one hot vectors
    enc = sklearn.preprocessing.OneHotEncoder(categories='auto')
    enc.fit(y.reshape(-1, 1))
    y = enc.transform(y.reshape(-1, 1)).toarray()

    # # save orignal y because later we will use binary
    # y_true = np.argmax(y_test, axis=1)
    print(f'x.shape: {x.shape}, y.shape: {y.shape}\n')
    if len(x.shape) == 2:  # if univariate
        # add a dimension to make it multivariate with one dimension 
        x = x.reshape((x.shape[0], x.shape[1], 1))

    input_shape = x.shape[1:]
    print(f'input_shape: {input_shape}\n')
    
    return x,y,input_shape

In [14]:
classifier_name="fcn"
root_dir="/data4/gsprivate/dl-4-tsc"
output_directory = root_dir + '/results/' + classifier_name + '/' + archive_name + '' + '/' + \
                    dataset_name + '/'
                    
if create_directory(output_directory) is None:
    print("Creating directory:{} None".format(output_directory))

x,y,input_shape=data_final_process(final_data,labels)
fcn_classifier = geng.create_classifier(classifier_name, input_shape, nb_classes, output_directory)
print(f"Classifier x.shape={x.shape} ,y.shape={y.shape}, input_shape={input_shape}, nb_classes={nb_classes}")

/data4/gsprivate/dl-4-tsc/results/fcn/MTS/Kailuan/ exist
Creating directory:/data4/gsprivate/dl-4-tsc/results/fcn/MTS/Kailuan/ None
x.shape: (129, 1075, 4), y.shape: (129, 2)

input_shape: (1075, 4)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1075, 4)]         0         
                                                                 
 conv1d (Conv1D)             (None, 1075, 128)         4224      
                                                                 
 batch_normalization (Batch  (None, 1075, 128)         512       
 Normalization)                                                  
                                                                 
 activation (Activation)     (None, 1075, 128)         0         
                                                                 
 conv1d_1 (Conv1D)           (None, 1075, 256)         1640

2023-12-27 14:58:10.233416: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_COMPAT_NOT_SUPPORTED_ON_DEVICE: forward compatibility was attempted on non supported HW
2023-12-27 14:58:10.233545: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: soft
2023-12-27 14:58:10.233556: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: soft
2023-12-27 14:58:10.233826: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 510.108.3
2023-12-27 14:58:10.233945: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 510.47.3
2023-12-27 14:58:10.233955: E tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:312] kernel version 510.47.3 does not match DSO version 510.108.3 -- cannot find working devices in this configuration


 chNormalization)                                                
                                                                 
 activation_1 (Activation)   (None, 1075, 256)         0         
                                                                 
 conv1d_2 (Conv1D)           (None, 1075, 128)         98432     
                                                                 
 batch_normalization_2 (Bat  (None, 1075, 128)         512       
 chNormalization)                                                
                                                                 
 activation_2 (Activation)   (None, 1075, 128)         0         
                                                                 
 global_average_pooling1d (  (None, 128)               0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 2)                 258       
          

In [15]:
classifier_name="cnn"
root_dir="/data4/gsprivate/dl-4-tsc"
output_directory = root_dir + '/results/' + classifier_name + '/' + archive_name + '' + '/' + \
                    dataset_name + '/'
                    
if create_directory(output_directory) is None:
    print("Creating directory:{} None".format(output_directory))

x,y,input_shape=data_final_process(final_data,labels)
cnn_classifier = geng.create_classifier(classifier_name, input_shape, nb_classes, output_directory)
print(f"Classifier x.shape={x.shape} ,y.shape={y.shape}, input_shape={input_shape}, nb_classes={nb_classes}")

/data4/gsprivate/dl-4-tsc/results/cnn/MTS/Kailuan/ exist
Creating directory:/data4/gsprivate/dl-4-tsc/results/cnn/MTS/Kailuan/ None
x.shape: (129, 1075, 4), y.shape: (129, 2)

input_shape: (1075, 4)

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1075, 4)]         0         
                                                                 
 conv1d_3 (Conv1D)           (None, 1069, 16)          464       
                                                                 
 batch_normalization_3 (Bat  (None, 1069, 16)          64        
 chNormalization)                                                
                                                                 
 re_lu (ReLU)                (None, 1069, 16)          0         
                                                                 
 max_pooling1d (MaxPooling1  (None, 534, 16)           0 

In [16]:
classifier_name="mcnn"
root_dir="/data4/gsprivate/dl-4-tsc"
output_directory = root_dir + '/results/' + classifier_name + '/' + archive_name + '' + '/' + \
                    dataset_name + '/'
                    
if create_directory(output_directory) is None:
    print("Creating directory:{} None".format(output_directory))

x,y,input_shape=data_final_process(final_data,labels)
mcnn_classifier = geng.create_classifier(classifier_name, input_shape, nb_classes, output_directory)
print(f"Classifier x.shape={x.shape} ,y.shape={y.shape}, input_shape={input_shape}, nb_classes={nb_classes}")

x.shape: (129, 1075, 4), y.shape: (129, 2)

input_shape: (1075, 4)

Classifier x.shape=(129, 1075, 4) ,y.shape=(129, 2), input_shape=(1075, 4), nb_classes=2


In [17]:
geng.fit_splits_for_mcnn(mcnn_classifier,x,y,epochs=200)
print("DONE")

fold 1: x_train_fold.shape=(103, 1075, 4), y_train_fold.shape=(103, 2)


Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 1053, 4)]            0         []                            
                                                                                                  
 input_4 (InputLayer)        [(None, 527, 4)]             0         []                            
                                                                                                  
 input_5 (InputLayer)        [(None, 351, 4)]             0         []                            
                                                                                                  
 input_6 (InputLayer)        [(None, 1049, 4)]            0         []                            
                                                                                            

/data4/conda_envs/gsprivate/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


KeyboardInterrupt: 

In [ ]:
# geng.fit_splits(cnn_classifier,x,y,batch_size=16,epochs=280)
# print('DONE')